<a href="https://colab.research.google.com/github/keeeehun/RL/blob/main/problems/p_REINFORCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

# Model

Q network로 사용할 Neural Network 모델입니다.

- 본 실습 때 사용할 CartPole 환경에서는 (4 -> 64 -> 128 -> 2)의 레이어 구조를 사용합니다.

    - action이 뽑힐 확률을 받아야 하므로 forward 함수 마지막에 softmax를 취해줍니다.

- get_action_logprob 함수를 통해 환경에 넣어줄 int형의 action과 action이 뽑힐 확률값(log probability)을 받을 수 있습니다.

    - discrete action space이므로 Categorical 함수를 통해 Categorical 분포를 만들어줍니다.

In [ ]:
class CartpolePolicy(nn.Module):
    def __init__(self, obs_dim, n_actions, device):
        super(CartpolePolicy, self).__init__()

        self.obs_dim = obs_dim
        self.n_actions = n_actions
        self.device = device

        self.l1 = nn.Linear(obs_dim, 64)
        self.l2 = nn.Linear(64, 128)
        self.l3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l3(x)
        x = F.softmax(x, dim=-1)

        return x

    def get_action_logprob(self, obs):
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float32)
        obs = obs.to(self.device)
        output = self.forward(obs)
        '''
        To do
        '''
        categorical = 
        action = 
        logprob = 

        return action.item(), logprob

# Train

buffer에 저장해놓은 logprob과 return data를 사용해서 모델을 업데이트 하는 함수입니다.

$$\theta_{t+1} \approx \theta_t + \alpha[\partial_{θ}log\pi_{\theta}(a|s)G_t]$$ 

In [ ]:
def train(logprobs, returns, optim):
    optim.zero_grad()
    # Cumulate gradients
    for ret, logprob in zip(returns, logprobs):
        '''
        To do
        '''
        j = 
        j.backward()
    optim.step()

# Matplotlib

Matplotlib을 사용해서 테스트 reward와 TD error를 그래프로 보여주는 함수입니다.

In [ ]:
def plot(rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    # plt.title('')
    plt.ylabel('reward')
    plt.xlabel('episode')
    plt.plot(rewards)
    plt.show()

# Main

트레이닝에 필요한 환경, 모델, loss, optimizer 등을 정의하는 코드입니다.

logprob과 return을 저장해주기 위한 buffer를 따로 만들어줍니다.

In [ ]:
# Save device (cpu or cuda)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Make env
env = gym.make("CartPole-v0")

# shape
obs_shape = env.observation_space.shape
act_shape = tuple([1]) # int to Tuple
obs_dim = obs_shape[0]
n_actions = env.action_space.n
buffer_size = 1000

# Define NN
policy_model = CartpolePolicy(obs_dim=obs_dim, n_actions=n_actions, device=device)
policy_model.to(device)

# Optimizer
optimizer = optim.Adam(policy_model.parameters())

# training parameters
total_step = 60000
batch_size = 32
gamma = 0.99

# list for saving results
rewards = []

# list for saving return & logprobs
logprobs = []
returns = []

REINFORCE 트레이닝을 실행하는 메인 루프입니다.

- total_step만큼 돌면서 매 스탭마다 Buffer에 logprob과 reward를 넣어줍니다.
- done 조건이 되면 저장된 reward로 각 시점마다 return 값을 계산해주고, model을 update 해줍니다.

In [ ]:
episode_reward = 0
best_test_reward = -999999
obs = env.reset()
for t in range(total_step):
    action, logprob = policy_model.get_action_logprob(obs)
    next_obs, rew, done, _ = env.step(action)
    logprobs.append(logprob)
    returns.append(rew)
    episode_reward += rew
    obs = next_obs

    if done:
        # Save the best model so far
        if best_test_reward <= episode_reward:
            best_test_reward = episode_reward
            torch.save(policy_model.state_dict(), "policy_model.pt")

        # complete returns
        for i in range(len(returns)-2, -1, -1):
            returns[i] += returns[i+1]*gamma
        rewards.append(episode_reward)
        train(logprobs, returns, optimizer)
        logprobs = []
        returns = []
        obs = env.reset()
        episode_reward = 0
    
    if t % 1000 == 0:
        plot(rewards)

In [ ]:
!apt update
!apt install xvfb
!pip install pyvirtualdisplay
!pip install gym-notebook-wrapper

In [ ]:
import gnwrapper
import gym

env = gnwrapper.LoopAnimation(gym.make('CartPole-v0'))

obs = env.reset()
for _ in range(500):
    obs, rew, done, _ = env.step(env.action_space.sample()) # Take random action
    env.render()
    if done:
        obs = env.reset()

env.display()

In [ ]:
policy_model.load_state_dict(torch.load("policy_model.pt"))

env = gnwrapper.LoopAnimation(gym.make('CartPole-v0'))

obs = env.reset()

for _ in range(500):
    action, _ = policy_model.get_action_logprob(obs)  # Take action from trained model
    env.render()
    obs, rew, done, _ = env.step(action)
    if done:
        obs = env.reset()

env.display()